### Overfitting

과적합은 학습 데이터에 치중하여 모델이 학습하는 현상으로 새로운 데이터에 대해서 대응을 못하는 문제이다. 따라서 딥러닝에서 가장 쉽게 접할 수 있는 유형이지만 개선하기 힘든 문제다.

#### 1. Dropout & Batch Normalization
- Dropout은 학습 중 일부 뉴런을 무작위로 비활성화해 과적합을 줄인다.
- Batch Normalization은 각 층의 출력값 분포를 정규화해 학습을 안정시키고 일반화 성능을 높인다.
- 둘 다 신경망이 특정 패턴에 과도하게 의존하지 않도록 돕는다.

In [ ]:
class CNN(nn.Module):
    def __init__(self): # 클래스 생성자 (초기 설정)
        super(CNN, self).__init__() # nn.Module의 생성자도 호출 (필수)
        
        # Feature Extraction 부분 (합성곱, 배치 정규화, 활성화, 풀링)
        self.feature_extraction = nn.Sequential(nn.Conv2d(3, 6, 5), # 3채널 입력 → 6채널 출력, 5x5 필터
                                                nn.BatchNorm2d(6), # 6채널에 대해 배치 정규화 (정규화로 학습 안정화)
                                                nn.ReLU(),
                                                nn.MaxPool2d(2, 2), 
                                                nn.Conv2d(6, 16, 5),
                                                nn.BatchNorm2d(16), # 16채널에 대해 배치 정규화
                                                nn.ReLU(),
                                                nn.MaxPool2d(2, 2))
        
        # Classifier 부분 (완전 연결 계층 + 드롭아웃)
        self.classifier = nn.Sequential(nn.Linear(512, 120),
                                        nn.ReLU(),
                                        nn.Dropout(0.5), # 비활성화 시킬 노드의 비율, 50% 확률로 뉴런 끄기 (Dropout으로 과적합 방지)
                                        nn.Linear(120, 64),
                                        nn.ReLU(),
                                        nn.Linear(64, 10))
    
    # 순전파 함수 정의                                    
    def forward(self, x): 
        x = self.feature_extraction(x) # 합성곱 레이어 통과
        x = x.view(-1, 512) # Flatten: 배치 크기 유지하며 펼치기
        x = self.classifier(x) # 분류기 통과

        return x

net = CNN().to(device) # 모델 선언후 MPS로 이동

#### 2. L2 Regularization
- 모델의 가중치가 너무 커지는 걸 방지하기 위해 가중치 크기에 패널티를 준다.
- 이는 복잡한 모델이 되지 않도록 제약을 걸어준다.
- 결과적으로 단순한 모델이 되도록 유도해 과적합을 완화한다.

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)
# weight_decay=1e-2
# L2 정규화로 가중치가 너무 커지는 걸 방지 (과적합 방지)

#### 3. Data Augmentation
- 기존 데이터를 회전, 자르기, 반전 등으로 변형해 학습 데이터를 다양하게 만든다.
- 이는 모델이 더 일반적인 패턴을 학습하게 도와준다.
- 특히 이미지나 음성 데이터에서 효과적이다.

In [ ]:
import torchvision.transforms as tr
import PIL

transf = tr.Compose(
                [tr.ToPILImage(), tr.RandomCrop(60), tr.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                 tr.RandomHorizontalFlip(),
                 tr.RandomRotation(10, resample=PIL.Image.BILINEAR),
                 tr.ToTensor()
                 ])

#### 4. Label Smoothing

Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, Zbigniew Wojna, Rethinking the Inception Architecture for Computer Vision, CVPR2016

각각의 클래스의 격차를 줄여주는 방법이다.

Label Smoothing은 정답에 100% 확신하지 않고 확률을 약간 분산시켜 과적합을 방지하는 기법이다.
예를 들어 정답이 [0, 1, 0]인 경우(원-핫 인코딩), smoothing=0.2를 적용하면 [0.1, 0.8, 0.1]처럼 변형된다.
이렇게 하면 모델이 정답(1)만 집착하지 않고 더 일반화된 예측을 하게 된다.

- 정답 라벨을 완전히 0 또는 1로 하지 않고 약간 부드럽게 조정한다.
- 이로 인해 모델이 너무 확신하는 예측을 하지 않게 된다.
- 결과적으로 일반화 성능이 좋아지고 overfitting이 줄어든다.


In [ ]:
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()

        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        # softmax는 값이 전체에 대해서 0~1 사이 값으로 정규화된 확룰 분포가 되도록한다.(총합1) 즉 각 클래스가 정답일 확률처럼 해석할 수 있게 해준다.
        # log_softmax는 softmax는 softmax를 계산한 다음에 log(자연로그)를 취하는 함수이다.
        # 후에 CrossEntropy Loss 계산을 잘하기 위해서 이다. loss = -∑ (true_prob * log(predicted_prob))
        pred = pred.log_softmax(dim=self.dim) # Cross Entropy 부분의 log softmax 미리 계산하기
        
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred) # 예측값과 동일한 크기의 영텐서 만들기
            true_dist.fill_(self.smoothing / (self.cls - 1)) # alpha/(K-1)을 만들어 줌(alpha/K로 할 수도 있음)
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) # (1-alpha)y + alpha/(K-1)
        
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim)) # Cross Entropy Loss 계산

In [ ]:
ls = LabelSmoothingLoss(10, smoothing=0.2) # 클래스 수 10, 부드럽게 0.2 적용

In [ ]:
nn.CrossEntropyLoss(label_smoothing=0.2) # pytorch가 업데이트되어 label smoothing 지원한다.